In [1]:
import numpy as np
import struct

In [2]:
def createMatrixList(len,rows,cols):
    MatrixList = np.zeros((len,rows,cols),dtype=np.int8)
    for i in range(len):
        for j in range(rows):
            for k in range(cols):
               MatrixList[i][j][k] = (i+1)*10+(j)*cols+(k+1)
    return MatrixList

In [ ]:
def writeMatrixList(filename,matrix_list):
    matrixData = [2051]
    matrixData.extend(matrix_list.shape)
    matrixData.extend(matrix_list.flatten())
    index = 0
    with open(filename,'wb') as f:
        for data in matrixData:
            if index < 4:
                s = struct.pack('>i', data)
                f.write(s)
            else:
                s = struct.pack('>B', data)
                f.write(s)
            index +=1

In [ ]:
def readMatrixFromFile(filename):
    data = open(filename, 'rb').read()
    off_set = 0
    header = '>iiii'
    magic_number, num_matrix, rows, cols = struct.unpack_from(header, data, off_set)
    off_set += struct.calcsize(header)
    fmt_Matrix = '>{}B'.format(num_matrix*rows*cols)
    matrix_list = struct.unpack_from(fmt_Matrix, data, off_set)
    matrix_list = np.reshape(matrix_list, (num_matrix, rows, cols))
    return matrix_list

In [ ]:
def trainBatchReader(idx_filename, batch_size, drop_last):
    data = open(idx_filename, 'rb').read()
    off_set = 0
    header = '>iiii'
    magic_number, num_matrix, rows, cols = struct.unpack_from(header, data, off_set)
    off_set += struct.calcsize(header)
    fmt_Matrix = '>{}B'.format(num_matrix*rows*cols)
    matrix_list = struct.unpack_from(fmt_Matrix, data, off_set)
    matrix_list = np.reshape(matrix_list, (num_matrix, rows, cols))
    num_batch= 0
    if drop_last:
        num_batch = num_matrix // batch_size
    else:
        if(num_matrix % batch_size == 0):
            num_batch = num_matrix // batch_size
        else:
            num_batch = num_matrix // batch_size + 1
    for i in range(num_batch):
        matrix_batch = matrix_list[batch_size*i:batch_size*(i+1)]
        yield list(matrix_batch)

In [ ]:
from CreateMatrix import createMatrixList

if __name__ == '__main__':
    test = input().split(' ')
    length,rows,cols=list(map(int, test))
    matrix_list = createMatrixList(length,rows,cols)
    print(matrix_list)

In [ ]:
import numpy as np
import CreateMatrix
import WriteMatrix
import ReadMatrix
if __name__ == '__main__':
    test = input().split(' ')
    length,rows,cols=list(map(int, test))
    matrix_list = CreateMatrix.createMatrixList(length,rows,cols)
    WriteMatrix.writeMatrixList('/data/workspace/myshixun/creatematrix/data/data.idx3.ubyte',matrix_list)
    matrix_list = ReadMatrix.readMatrixFromFile('/data/workspace/myshixun/creatematrix/data/data.idx3.ubyte')
    print(type(matrix_list))
    print(matrix_list)

In [ ]:
import numpy as np
import struct
from CreatMatrix import createMatrixList
from WriteMatrix import writeMatrixList
from BatchReader import trainBatchReader

if __name__ == '__main__':
    *test,batch,drop = input().split(' ')
    print(test,batch,drop)
    length,rows,cols=list(map(int, test))
    matrix_list = createMatrixList(length,rows,cols)
    writeMatrixList('/data/workspace/myshixun/creatematrix/data/data.idx3.ubyte',matrix_list)
    reader = trainBatchReader(idx_filename='/data/workspace/myshixun/creatematrix/data/data.idx3.ubyte',batch_size=int(batch), drop_last=(drop=='True'))
    print(type(reader))
    for id, data in enumerate(reader):
        print(id,data)

In [ ]:
import numpy as np
import struct
import CreateMatrix
from WriteMatrix import writeMatrixList
type2code_dict = {'uint8':0x08,'int8':0x09,'int16':0x0B,'int32':0x0C,'float32':0x0D,'float64':0x0E}
code2type_dict = {0x08: 'B', 0x09: 'b', 0x0B: 'h', 0x0c: 'i', 0x0D: 'f', 0x0E: 'd'}

def readMatrixFromFile(filename):
    with open(filename,'rb') as f:
        data_buf = f.read()
        off_set = 0
        file_head_fmt = '>HBB'#格式定义，>表示高位在前，I表示4字节整数
        _,dcode,dimslen = struct.unpack_from(file_head_fmt,data_buf,off_set)
        off_set += struct.calcsize(file_head_fmt)

        file_head_fmt = '>{}I'.format(dimslen)
        shapes = struct.unpack_from(file_head_fmt,data_buf,off_set)
        off_set += struct.calcsize(file_head_fmt)

        data_fmt = '>'+ str(np.prod(shapes)) + code2type_dict[dcode]
        matrix_list = struct.unpack_from(data_fmt,data_buf,off_set)
        matrix_list = np.reshape(matrix_list,shapes)

    return matrix_list
    
if __name__ == '__main__':
    matrix_list = CreateMatrix.createMatrixList(len=10,rows=2,cols=3)
    writeMatrixList('/data/workspace/myshixun/creatematrix/data/data.idx3.ubyte',matrix_list)
    print('read')

    matrix_list = readMatrixFromFile('/data/workspace/myshixun/creatematrix/data/data.idx3.ubyte')
    print(matrix_list)